In [1]:
%load_ext autoreload
%autoreload 2

import pennylane as qml
import math
from pennylane import qaoa
from pennylane import numpy as np
import matplotlib.pyplot as plt
from vqa.hamiltonian.protein_folding import protein_folding_hamiltonian
from vqa.hamiltonian.Protein_models.CoordinateBased_HPLattice import CoordinateBased_HPLattice
from vqa.utils.protein_utils import *
from collections import Counter

In [2]:
# Make an instance for the protein
sequence = [1,0,0,1] #HPPH
L1 = 2
L2 = 2

lambda_vector = (2.1, 2.4, 3)

protein = CoordinateBased_HPLattice((L1, L2), sequence = sequence, lambda_vector = lambda_vector)
print(protein)
protein.calc_solution_sets()
feasible_set = np.array(protein.feasible_set)


O:
[-2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1]
T:
[[ 0.   4.2  0.   0.   2.4  0.  -1.  -1. ]
 [ 4.2  0.   0.   0.   0.   2.4 -1.  -1. ]
 [ 0.   0.   0.   4.2  0.   0.   2.4  0. ]
 [ 0.   0.   4.2  0.   0.   0.   0.   2.4]
 [ 2.4  0.   0.   0.   0.   4.2  0.   0. ]
 [ 0.   2.4  0.   0.   4.2  0.   0.   0. ]
 [-1.  -1.   2.4  0.   0.   0.   0.   4.2]
 [-1.  -1.   0.   2.4  0.   0.   4.2  0. ]]
Dn:
[2, 2, 2, 2]


In [3]:
# Make into Hamiltonian
H_cost = protein_folding_hamiltonian(protein)
num_qubits = protein.num_bits
qubits = range(protein.num_bits)

H_mixer = qaoa.x_mixer(qubits)

In [85]:
num_of_samps = 100
dev = qml.device('default.qubit', wires = qubits, shots = num_of_samps)

# Put the Mixer and Cost Hamiltonians in a layer so that we can repeat it
def qaoa_layer(gamma, beta):
    qaoa.cost_layer(gamma, H_cost)
    qaoa.mixer_layer(beta, H_mixer)


# Then repeat it in a circuit with starting in a super position of all bitstrings
def circuit(params):     # Gamma and Beta values can be put together to be an array of parameters
    for q in qubits:     # To start in a superposition we place a Hadamard on all qubits
        qml.Hadamard(wires = q)
    qml.layer(qaoa_layer, len(params[0]), params[0], params[1])
    return qml.sample()

    


#def samp_energies(params):
    #circuit(params)
    
    

SyntaxError: invalid syntax (3074512965.py, line 16)

In [83]:
@qml.qnode(dev)
def CVaR(params):
    
    _set = circuit(params)

    energies = energies_of_set(_set, H_cost, num_qubits)

    sorted_energy_samples = np.sort(energies)
    
    
    summa = sum(sorted_energy_samples[0:int(alpha*num_of_samps)])/(alpha*num_of_samps)
    #for i in range(int(alpha*num_of_samps)):
    #    summa += sorted_energy_samples[i]
    #summa = summa/(alpha*num_of_samps)
    return float(summa)



In [84]:
params = [[0.5],[0.5]]

alpha=0.1

CVaR(params)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'SampleMP'

In [78]:

#param_shape = StronglyEntanglingLayers.shape(n_layers=len(params[0]), n_wires=qubits)
#init_params = np.random.uniform(low=0, high=2*np.pi, size=param_shape, requires_grad=True)

eta = 0.01
steps = 100
alpha = 0.1

qnode_stochastic = qml.QNode(CVaR, dev, interface="autograd")


dev.shots = 100
cost_SGD100 = []
params_SGD100 = params
opt = qml.GradientDescentOptimizer(eta)

qnode_stochastic(params)


for _ in range(steps):
    cost_SGD100.append(qnode_stochastic(params_SGD100))
    params_SGD100 = opt.step(qnode_stochastic, params_SGD100)

QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.